In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import firm_revised


In [2]:
df = pd.read_csv(f'../data/market_integrates_100.csv')
print(df)

    time  product  characteristic1  characteristic2      price  marginal_cost  \
0      1        1         5.543405         5.891322  12.316490       7.120157   
1      1        2         5.278369         5.209202  13.802273       8.193672   
2      1        3         5.424518         5.185328  13.915652       8.278634   
3      1        4         5.844776         5.108377  11.729217       6.690549   
4      1        5         5.004719         5.219697  15.818798       9.660586   
5      1        6         5.121569         5.978624  14.447553       8.669650   
6      1        7         5.670749         5.811683  11.552733       6.565409   
7      1        8         5.825853         5.171941  14.604947       8.787955   
8      1        9         5.136707         5.816225  15.033567       9.096175   
9      1       10         5.575093         5.274074  13.587893       8.042679   
10     2        1         5.543405         5.891322  16.230208       9.767099   
11     2        2         5.

In [5]:
# All the parameters of interest 
beta = np.array([2, -0.5, -0.3])
mu = 0.5
omega = 1


# The number of firms in the market
J = 10

# The number of product characteristics 
K = 2

# The number of consumers in the market 
N = 500

# The number of period that I want to run the market for 
T = 100

In [7]:
# Unpacking the X and the price
X1 = np.array(df['characteristic1'][:J])
X2 = np.array(df['characteristic2'][:J])
all_X = np.column_stack((X1, X2))
X0 = np.ones(J)
X = np.column_stack((X0, all_X))

eq_price = np.array(df['price'])

true_share = np.array(df['mshare'])
# print(true_share)

all_est_shares = np.zeros(J*(T-1))
# print(all_est_shares)


In [8]:
# The idea is that you want to min the distance between the true share and the made up share by optimizing the fiven vector theta that determines the share 

def f(theta, true_share, N, J, X, price):
    beta = theta[:K+1]
    mu = theta[K+1]
    omega = theta[K+2]  
    # make a numpy array all estimated shares 
    for t in range(0, T-1):
        eq_price_slice = price[t*J:(t+1)*J]
        v_p = np.random.normal(0, 1, N)
        e = np.random.gumbel(0, 1, N*J)
        shares_per_firm = firm_revised.share(N, J, X, v_p, eq_price_slice, beta, mu, omega, e)[0]
        all_est_shares[t*J:(t+1)*J] = shares_per_firm
    return np.linalg.norm(true_share - all_est_shares)

In [9]:
theta_0 = np.array([2., -0.5, -0.3, 0.5, 1.])
# theta_0 = np.array([1.5, -1., -1., 0.4, 1.])


In [10]:
res = minimize(f, theta_0, args=(true_share, N, J, X, eq_price), method = 'Nelder-Mead')
optim_vec = res.x
print(optim_vec)

ValueError: cannot reshape array of size 0 into shape (1,10)

In [ ]:
all_val_b0 = []
all_val_b1 = []
all_val_b2 = []
all_val_mu = []
all_val_omega = []

for l in range(1, 10):
    
    res = minimize(f, theta_0, args=(true_share, N, J, X, eq_price), method = 'Nelder-Mead')
    optim_vec = res.x

    all_val_b0.append(optim_vec[0])
    all_val_b1.append(optim_vec[1])
    all_val_b2.append(optim_vec[2])
    all_val_mu.append(optim_vec[3])
    all_val_omega.append(optim_vec[4])

print(all_val_b0)
print(all_val_b1)
print(all_val_b2)
print(all_val_mu)
print(all_val_omega)


[1.5445294620652326, 1.533597011184653, 1.5000925016385158, 1.5008345213932603, 1.5248478589690415, 1.5553369736872442, 1.5089471290243461, 1.4636734360165824, 1.4999386312406822]
[-1.003864589295763, -0.996836904534302, -0.9987509236670302, -1.0025761284204953, -1.0132299223453436, -0.9877159790633792, -0.9913833686970374, -1.0331552991090085, -0.9999039064897062]
[-1.025855090764588, -1.0020146894209225, -0.9993761823400188, -0.9954613587004998, -0.9624035000143596, -0.9535323710718517, -1.0059491588241476, -0.9951074960721809, -1.0002130518245171]
[0.38465081741745066, 0.39956111353293844, 0.3999274560731221, 0.4182669810881973, 0.40803511278011473, 0.4139409224550189, 0.4063849796476491, 0.4139521031268942, 0.41997384714914177]
[1.0214688222673305, 1.0301909001397576, 1.0499669901478823, 1.0205079689705037, 1.0249292318502468, 1.0338105239124982, 1.0247585963241475, 1.0307462752440417, 0.9998706097808063]


In [ ]:
df = pd.DataFrame({'b0': all_val_b0,
                   'b1': all_val_b1,
                   'b2': all_val_b2, 
                   'mu': all_val_mu, 
                   'omega': all_val_omega,  
                   })
df.to_csv(f'../data/estimation_market_{seed}.csv', index=False)
print(df)


         b0        b1        b2        mu     omega
0  1.544529 -1.003865 -1.025855  0.384651  1.021469
1  1.533597 -0.996837 -1.002015  0.399561  1.030191
2  1.500093 -0.998751 -0.999376  0.399927  1.049967
3  1.500835 -1.002576 -0.995461  0.418267  1.020508
4  1.524848 -1.013230 -0.962404  0.408035  1.024929
5  1.555337 -0.987716 -0.953532  0.413941  1.033811
6  1.508947 -0.991383 -1.005949  0.406385  1.024759
7  1.463673 -1.033155 -0.995107  0.413952  1.030746
8  1.499939 -0.999904 -1.000213  0.419974  0.999871
